In [2]:
!pip install openai --quiet
!pip install langchain --quiet
!pip install cohere --quiet
!pip install tiktoken --quiet
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.4/323.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [11]:
from google.colab import userdata
import os
# os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
os.environ['COHERE_API_KEY'] = userdata.get("Cohere_API")

In [4]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [5]:
!pip install pypdf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.0/331.0 kB 6.3 MB/s eta 0:00:00


In [6]:
from langchain_community.document_loaders import PyPDFLoader

In [7]:
loader = PyPDFLoader("EMPLOYEE_AGREEMENT.pdf")
pages = loader.load()
print(len(pages))

13


In [8]:
full_text =""
for page in pages:
  full_text += page.page_content

print("Pages", len(pages))
print("Lines" , len(full_text.split("\n")))
print("Words" , len(full_text.split(" ")))
print("Charecters", len(full_text))

Pages 13
Lines 369
Words 4204
Charecters 29540


# Step-2:Split the data into Chunks

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

133


In [11]:
print(chunks[0])

page_content='EX-10.1 3 smtp_ex10z1.htm EMPLOYEE AGREEMENT
EXHIBIT 10.1
EMPLOYEE AGREEMENT
THIS EMPLOYEE AGREEMENT made as of September___, 2014, by and between SharpSpring,
Inc., a Delaware corporation (the “Company”), whose principal place of business is at 802 NW 5th Avenue,' metadata={'producer': 'Skia/PDF m123', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'creationdate': '2024-04-03T12:51:00+00:00', 'title': 'EMPLOYEE AGREEMENT', 'moddate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}


# Step-3: Creating embeddings

In [12]:
!pip install -U langchain-openai
!pip install -U langchain-cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.6 MB/s eta 0:00:00


In [13]:
from langchain_openai import OpenAIEmbeddings
from langchain_cohere import CohereEmbeddings

In [14]:
#embeddings = OpenAIEmbeddings()
from langchain_cohere import CohereEmbeddings
import os


embeddings = CohereEmbeddings(
    model="embed-english-v3.0"   # REQUIRED
)

# Test embedding
sample_embedding = embeddings.embed_query("You must follow the rules")
print(len(sample_embedding))
print(sample_embedding)


1024
[0.08300781, -0.00831604, -0.0259552, -0.06921387, 0.0015554428, -0.053253174, 0.012184143, 0.014343262, -0.034423828, 0.011711121, -0.017456055, 0.03717041, -0.029754639, -0.025772095, 0.047668457, -0.054229736, 0.005821228, 0.021209717, -0.005405426, -0.025741577, 0.039123535, -0.009216309, -0.047668457, 0.09539795, -0.008285522, 0.023468018, 0.037597656, 0.0012578964, -0.009109497, -0.010696411, 0.013221741, -0.016906738, 0.01928711, 0.05593872, -0.06713867, 0.040740967, -0.013977051, 0.0044174194, -0.008636475, 0.037902832, 0.04901123, -0.0046958923, -0.044281006, 0.010139465, -0.057861328, -0.049713135, 0.042236328, 0.0026893616, 0.011802673, 0.036499023, 0.029129028, -0.0006132126, 0.026977539, -0.014427185, -0.0064811707, -0.0029697418, 0.0030498505, -0.046325684, 0.0473938, -0.02217102, -0.009841919, -0.03451538, 0.05355835, -0.0045814514, 0.053100586, 0.002368927, -0.02420044, 0.012329102, 0.0062408447, 0.04119873, 0.017715454, -0.07775879, 0.025985718, -0.038909912, 0.00

In [15]:

sample_docs=["You must follow the rules",
            "You must not disclose the rules"]

embeded_vectors1= embeddings.embed_documents(sample_docs)
print(embeded_vectors1)
print(len(embeded_vectors1))
print(len(embeded_vectors1[0]))


[[0.07110596, 0.0059547424, -0.01322937, -0.08050537, 0.011558533, -0.040222168, -0.029327393, -0.0126571655, -0.023422241, 0.022750854, 0.0024700165, 0.012901306, -0.036376953, -0.022109985, 0.038879395, -0.03704834, 0.017929077, 0.009391785, -0.01209259, -0.02619934, 0.00082969666, 0.0006737709, -0.054229736, 0.07165527, 0.0107421875, 0.011260986, 0.032348633, -0.0050964355, 0.0032444, -0.012367249, 0.004421234, -0.0020122528, 0.006465912, 0.031280518, -0.053710938, 0.045654297, -0.022399902, 0.011230469, -0.022003174, 0.051727295, 0.02822876, -0.006500244, -0.059326172, -0.007411957, -0.080444336, -0.04547119, 0.037475586, 0.015388489, 0.004081726, 0.024246216, 0.004825592, -0.020004272, 0.013061523, -0.0011978149, -0.014656067, -0.0048675537, -0.009895325, -0.051483154, 0.027145386, -0.01902771, -0.0028743744, -0.033966064, 0.066345215, 0.022277832, 0.033355713, -0.0044555664, -0.021697998, 0.024108887, 0.003282547, 0.03479004, -0.003900528, -0.06774902, 0.011947632, -0.022491455, 

# Step-4: Storing in Vector Stores

In [16]:
!pip install chromadb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently

In [17]:
from langchain_community.vectorstores import Chroma
emp_rules_db= Chroma.from_documents(chunks,
                                    embeddings,
                                    persist_directory="emp_rules_db"
                          )
emp_rules_db.persist()
#persist() stores teh DB Locally on Hard-disk insted of RAM

/tmp/ipython-input-175680650.py:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  emp_rules_db.persist()


# Step-5: Retrieval

In [18]:
retriever = emp_rules_db.as_retriever()
result=retriever.invoke("What is the policy for sick leaves")
result

[Document(metadata={'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'page_label': '2', 'total_pages': 13, 'title': 'EMPLOYEE AGREEMENT', 'creationdate': '2024-04-03T12:51:00+00:00', 'page': 1, 'moddate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'producer': 'Skia/PDF m123'}, page_content='sick days and personal absence days as authorized by the Company for its other employees.\nARTICLE 6\nFringe Benefits\nEmployee shall be entitled to participate in and receive benefits from all of the Company’s employee\nbenefit plans that are now, or in the future may be, maintained by the'),
 Document(metadata={'source': 'EMPLOYEE_AGREEMENT.pdf', 'moddate': '2024-04-03T12:51:00+00:00', 'title': 'EMPLOYEE AGREEMENT', 'producer': 'Skia/PDF m123', 'total_pages': 13, 'page_label': '2', 'page': 1, 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0

In [19]:
for i in range(len(result)):
  print(result[i].metadata)

{'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'page_label': '2', 'total_pages': 13, 'title': 'EMPLOYEE AGREEMENT', 'creationdate': '2024-04-03T12:51:00+00:00', 'page': 1, 'moddate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'producer': 'Skia/PDF m123'}
{'source': 'EMPLOYEE_AGREEMENT.pdf', 'moddate': '2024-04-03T12:51:00+00:00', 'title': 'EMPLOYEE AGREEMENT', 'producer': 'Skia/PDF m123', 'total_pages': 13, 'page_label': '2', 'page': 1, 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'creationdate': '2024-04-03T12:51:00+00:00'}
{'source': 'EMPLOYEE_AGREEMENT.pdf', 'title': 'EMPLOYEE AGREEMENT', 'moddate': '2024-04-03T12:51:00+00:00', 'page_label': '2', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'total_pages': 13, 'creationdate'

In [22]:
retriever = emp_rules_db.as_retriever()
result=retriever.invoke("Employee Monthly Salary ")
pretty_print_docs(result)

Document 1:

Compensation of Employee
4.1. Base Compensation.  For all services rendered by Employee under this Employee Agreement, the
Company agrees to pay Employee the rate of $14,583 per month (the “base salary”), which shall be payable
----------------------------------------------------------------------------------------------------
Document 2:

to Employee not less frequently than bi-monthly, or as is consistent with the Company’s practice for its other
employees.  
4.2. Other Compensation.  Employee shall receive other compensation as more fully described on
Appendix B, attached hereto and made a part hereof.
----------------------------------------------------------------------------------------------------
Document 3:

Company for its employees, including, without limitation, the Company’s health insurance plan. No amounts
paid to Employee from an employee benefit plan shall count as compensation due Employee as base salary or
------------------------------------------------

In [23]:
for i in range(len(result)):
  print(result[i].metadata)

{'creationdate': '2024-04-03T12:51:00+00:00', 'page_label': '2', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'title': 'EMPLOYEE AGREEMENT', 'page': 1, 'producer': 'Skia/PDF m123', 'moddate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'total_pages': 13}
{'moddate': '2024-04-03T12:51:00+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'page_label': '2', 'creationdate': '2024-04-03T12:51:00+00:00', 'total_pages': 13, 'title': 'EMPLOYEE AGREEMENT', 'page': 1, 'producer': 'Skia/PDF m123'}
{'creationdate': '2024-04-03T12:51:00+00:00', 'title': 'EMPLOYEE AGREEMENT', 'moddate': '2024-04-03T12:51:00+00:00', 'page': 2, 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'source': 'EMPLOYEE_AGREEMENT.pd

**MultiQuery Retrival**

In [24]:
!pip install wikipedia -q

  Preparing metadata (setup.py) ... done


In [31]:
# Loaders
from langchain_community.document_loaders import WikipediaLoader

loader = WikipediaLoader(query="MS Dhoni")
documents = loader.load()

# Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

docs = text_splitter.split_documents(documents)

# Embeddings
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    cohere_api_key=os.environ["COHERE_API_KEY"],
    model="embed-english-v3.0"
)

# Vector DB
from langchain_community.vectorstores import Chroma

embeddings_db = Chroma.from_documents(
    docs,
    embeddings,
    persist_directory="wiki_db"
)

embeddings_db.persist()

print("Vector DB created successfully ✅")

Vector DB created successfully ✅


In [7]:
pip install --upgrade --force-reinstall langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.4/501.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 

In [20]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WikipediaLoader(query="MS Dhoni")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

docs = text_splitter.split_documents(documents)

In [22]:
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import Chroma
import os

embeddings = CohereEmbeddings(
    cohere_api_key=os.environ["COHERE_API_KEY"],model="embed-english-v3.0"
)

embeddings_db = Chroma.from_documents(
    docs,
    embeddings,
    persist_directory="wiki_db"
)

In [23]:
from langchain_classic.retrievers import MultiQueryRetriever
from langchain_cohere import ChatCohere

llm = ChatCohere(
    temperature=0,
    cohere_api_key=os.environ["COHERE_API_KEY"]
)

llm_based_retriever = MultiQueryRetriever.from_llm(
    retriever=embeddings_db.as_retriever(),
    llm=llm
)

question1 = "What is the DOB of Dhoni?"
question2 = "What Sport does Dhoni Play?"

rel_docs1 = llm_based_retriever.invoke(question1)
rel_docs2 = llm_based_retriever.invoke(question2)

print(len(rel_docs1), len(rel_docs2))

14 8


In [24]:
print("\n--- Results for Question 1 ---\n")
for i, doc in enumerate(rel_docs1):
    print(f"Document {i+1}:\n")
    print(doc.page_content[:500])  # first 500 characters
    print("\n" + "="*80 + "\n")

print("\n--- Results for Question 2 ---\n")
for i, doc in enumerate(rel_docs2):
    print(f"Document {i+1}:\n")
    print(doc.page_content[:500])
    print("\n" + "="*80 + "\n")


--- Results for Question 1 ---

Document 1:

== References ==


Document 2:

== References ==


Document 3:

== Early life ==
Dhoni was born on 7 July 1981 in Ranchi, Bihar (now in Jharkhand) in a Hindu Rajput family to Pan Singh and Devaki Devi. His parents hailed from Lwali village in the Almora district of Uttar Pradesh (now Uttarakhand). He was the youngest of three children. His family spells the surname as "Dhauni". The spelling "Dhoni" emerged due to a spelling mistake in his school certificates and, despite repeated attempts by his family, has never been rectified.


Document 4:

In 2008, Dhoni was awarded India's highest sport honour Major Dhyan Chand Khel Ratna Award by Government of India. He received the fourth highest civilian award Padma Shri in 2009 and third highest civilian award Padma Bhushan in 2018. Dhoni holds an honorary rank of Lieutenant colonel in the Parachute Regiment of the Indian Territorial Army which was presented to him by the Indian Army in 2011. In Ju